In [1]:
import scipy
import numpy as np
import os
import os.path
import networkx as nx
import networkx.drawing
import matplotlib.pyplot as plt
import sys
import math
from collections import Counter
import time
import pickle
import ast

import ase
from ase.io import read
from ase.build import molecule
from ase.geometry.analysis import Analysis as ana
import ase.neighborlist
from ase import Atoms

# np.set_printoptions(threshold=np.inf)

In [2]:
####### BASIC INFO #######

# This script loads a list of pairs of xyz files (see line with $$$$) and generates coupling_frag objects for each molecule.
# Coupling products are then generated using the coupling_frag objects and the output info is printed to coupling_output.txt.
# The output coupling products are labeled as:
#    [fragment name A]_[fragment name B]_[index of atom coupled on A]_[index of atom coupled on B]_[index of dimer generated from fragments A and B]
# for each coupling product, 3 files are generated
# an .xyz file with the geometry of the coupling product
# an .atl (atom list) file with the indices of the atoms belonging to each fragment in the coupling product
# a .dih (dihedral) file containing the four atom indices which describe the dihedral formed around the new bond

# Possible things to change to the script, depending on the use of this script:
# 1. bondlength formed (see line with !!!!), default is 1.35 A for Csp2-Csp2 distances
# 2. cutoff for what is considered a steric 'clash' between fragments (see lines with %%%%), default is 1.5 A
# 3. start search for optimal dihedral between fragments at either coplanar or orthogonal geometries (see lines with &&&&)

# requires python3 to run (here using anaconda/2019.10/python-3.7)

In [3]:
####### FUNCTIONS #######

In [4]:
def read_geom(xyz_file: str):

    """
    Parse the xyz file to assign the fragment name, atom types, and cartesian coordinates.

    :param xyz_file: path to the xyz file to parse

    :return atoms: ordered list of element types for each atom
    :return coord: cartesian coordinates of the molecule (2D array)
    :return n_atoms: number of atoms in the molecule
    :return name: name of the molecule
    """

    try:
        assert xyz_file[-4:] == ".xyz"
        xyz = open(xyz_file, "r")
    except:  # raises error and exits if xyz file requested isn't found
        raise FileNotFoundError("Could not read xyz file: {0}".format(xyz_file))

    atoms = []
    coord = []
    # takes name of xyz as name of compound, strips ".xyz" and anything before and including last slash (in path)
    name = xyz_file.rsplit("/", 1)[-1][:-4]

    try:
        n_atoms = int(xyz.readline())  # first line in xyz file: number of atoms
        next(xyz)  # skip the second line of the xyz, which could be anything
        for line in xyz:  # all the remaining lines, atoms and cartesian coordinates
            line_data = (
                line.split()
            )  # extra condition, in case there are empty lines at the end of the file
            if (
                len(line_data) == 4
            ):  # only goes through the remaining lines with an atom and three coordinates
                atom, x, y, z = line.split()
                atoms.append(str(atom))
                coord.append([float(x), float(y), float(z)])

        assert n_atoms == len(coord)

    except:
        raise AssertionError("Could not parse xyz file: {0}".format(xyz_file))

    xyz.close()
    coord = np.asarray(coord)

    return atoms, coord, n_atoms, name

In [5]:
def genconmat_ase(xyz_file: str, factor=1.0):

    """
    Generate connectivity, laplacian matrices and number of connections per atom for a molecule using ASE.

    :param xyz_file: path to the xyz file to parse
    :param factor: value by with the sum of covalent radii of atom pairs is multiplied to determine bonding (1.0 is best for ASE)

    :return conmat: connectivity matrix of the molecule (2D array)
    :return connec: number of connections per atom (1D array)
    :return laplacian: laplacian of the connectivity matrix of the molecule (2D array)
    """

    ending = xyz_file.strip()[-3:]  # find file ending (should be xyz)
    structure = read(
        xyz_file, format=ending
    )  # use ase to read xyz file and save molecule as an 'Atoms' format

    # atnum = structure.get_atomic_numbers() # extra lines if we want to generate atnum, atoms and n_atoms directly from ASE
    # atoms = structure.get_chemical_symbols()
    # n_atoms = len(atlist)

    # define the cutoff to add to the covalent radii of the atoms with a multiplying factor
    # using mult = 1.2 or above finds more bonding than we expect
    cutoff = ase.neighborlist.natural_cutoffs(structure, mult=factor)

    # generate neighbor list; 'self-interaction' if the atom forms a bond with itself (useless)
    # 'bothways' if both atoms are considered to be bonded
    nl = ase.neighborlist.NeighborList(cutoff, self_interaction=False, bothways=True)
    nl.update(structure)

    conmat = nl.get_connectivity_matrix(sparse=False)  # build connectivity matrix
    conmat = conmat.astype(float)

    connec = np.zeros_like(
        conmat
    )  # make a matrix of zeroes in the same shape as conmat
    for atm in range(len(conmat)):
        connec[atm, atm] = len(
            np.where(conmat[atm, :] == 1.0)[0]
        )  # collect the sum of each row of the connectivity matrix and place that along the diagonal of the d matrix
    laplacian = (
        connec - conmat
    )  # generate laplacian matrix by putting the d matrix as the diagonal of the connectivity matrix
    # laplacian = laplacian.astype(float)

    connec = connec.diagonal()  # just take diagonal elements
    # connec = connec.astype(float)

    return conmat, connec, laplacian

In [6]:
def genradii(elems: list):

    """
    Save the covalent radius and atomic number for each atom. Covalent radii drawn from Cordero et al. "Covalent radii revisited". Dalton Trans. 2008, 21, 2832.
    In some cases, smaller radii are given, because they lead to fewer errors.

    :param elems: list of element types for each atom in the molecule

    :return radii: ordered list of the covalent radius for each atom
    :return atnum: ordered list of atomic numbers for each atom
    """

    rad = []
    atnum = []
    for elem in elems:  # the most common elements are put earliest
        if elem == "C":
            rad.append(0.68), atnum.append(
                6.0
            )  # normally 0.76 for single bond, sergi found 0.68 works better
        elif elem == "H":
            rad.append(0.23), atnum.append(
                1.0
            )  # normally 0.31, sergi found 0.23 works better
        elif elem == "O":
            rad.append(0.73), atnum.append(
                8.0
            )  # normally 0.66 for single bond, sergi found 0.73 works better
        elif elem == "N":
            rad.append(0.75), atnum.append(
                7.0
            )  # normally 0.71 for single bond, sergi found 0.75 works better
        elif elem == "S":
            rad.append(1.02), atnum.append(16.0)  # normally 1.05
        elif elem == "P":
            rad.append(1.06), atnum.append(15.0)  # normally 1.07
        elif elem == "F":
            rad.append(0.71), atnum.append(9.0)  # normally 0.57
        elif elem == "Cl":
            rad.append(0.99), atnum.append(17.0)  # normally 1.02
        elif elem == "Br":
            rad.append(1.21), atnum.append(35.0)  # normally 1.20
        elif elem == "I":
            rad.append(1.40), atnum.append(53.0)  # normally 1.39
        elif elem == "B":
            rad.append(0.82), atnum.append(5.0)  # normally 0.84
        elif elem == "Si":
            rad.append(1.11), atnum.append(14.0)  # normally 1.11
        elif elem == "Se":
            rad.append(1.16), atnum.append(34.0)  # normally 1.20
        elif elem == "Te":
            rad.append(1.35), atnum.append(52.0)  # normally 1.38
        elif elem == "As":
            rad.append(1.19), atnum.append(33.0)  # normally 1.19
        else:
            print("Element out of range: {0}".format(elem))

    return rad, atnum

In [7]:
def genconmat(
    n_atoms: int,
    atoms: list,
    coord: np.ndarray,
    radii: list,  # covalent radii of each atom
    factor=1.3,
):

    """
    Generate connectivity, laplacian matrices and number of connections per atom for a molecule using a homemade script.

    :param n_atoms: number of atoms in the molecule
    :param atoms: ordered list of element types for each atom
    :param coord: cartesian coordinates of the molecule (2D array)
    :param radii: list of the covalent radius for each atom
    :param factor: value by with the sum of covalent radii of atom pairs is multiplied to determine bonding (1.3 is best for the radii given in gen_radii)

    :return conmat: connectivity matrix of the molecule (2D array)
    :return connec: number of connections per atom (1D array)
    :return laplacian: laplacian of the connectivity matrix of the molecule (2D array)
    """

    conmat = np.zeros((n_atoms, n_atoms))

    for i in range(0, n_atoms):  # loop through cartesian coordinates of all heavy atoms
        for j in range(i, n_atoms):
            if i != j:
                a = np.array(coord[i])
                b = np.array(coord[j])
                dist = np.linalg.norm(a - b)  # Euclidean distance between the two atoms
                thres = (
                    radii[i] + radii[j]
                ) * factor  # multiply the sum of covalent radii by fudge factor

                if dist <= 0.2:  # only if two atoms are too close, print a warning
                    print(
                        "Atom {0} {1} is overlapped with atom {2} {3}".format(
                            str(i + 1), labels[i], str(j + 1), labels[j]
                        )
                    )
                elif (
                    dist <= thres
                ):  # otherwise place a 1 indicating connectivity between two atoms
                    conmat[
                        i, j
                    ] = 1  # if they are at a distance smaller than the sum of their radii
                    conmat[j, i] = 1
                else:
                    pass  # if atoms are too distant, leave connectivity between these two atoms at 0

    connec = np.zeros((n_atoms))  # the connectivity per atom

    for i in range(0, n_atoms):
        connec[i] = np.sum(
            conmat[i, :]
        )  # sum all elements in each column to get the conectivity of each atom

    degree = np.diag(
        connec
    )  # make a diagonal matrix containing the connectivity per atom
    laplacian = degree - conmat  # generate laplacian matrix using this expression

    return conmat, connec, laplacian

In [8]:
def identify_coupling_sites(
    atoms: list,  # elements in compound
    conmat: np.ndarray,  # connectivity matrix
    connec: np.ndarray,  # number of connections per atom
):

    """
    Read the connectivity of the molecule and applies simple chemical rules to determine
    any available sp2 coupling sites on C or N.

    :param conmat: connectivity matrix (2D array)
    :param atoms: ordered list of element types for each atom
    :param connec: number of connections per atom (1D array)

    :return coupling_sites: list of tuples, each containing the indices (heavy atom to couple, hydrogen to remove, adjacent heavy atom)
    """

    coupling_sites = []

    for atom_ind, atom in enumerate(atoms):
        # atoms that are sp2/sp carbons or sp2 nitrogens
        if ((atom == "C") and (2.0 <= connec[atom_ind] <= 3.0)) or (
            (atom == "N") and (connec[atom_ind] == 2.0)
        ):
            # makes an array where the first element is a list of the indices where '1' is found (second element is empty)
            bond_locs = np.where(conmat[atom_ind] == 1.0)

            # dict. where keys are indices of atoms bonded to the sp/sp2 coupling atom and values are their element type
            # and save there the atoms found in the connectivity matrix to be bound to that atom
            bonded_atoms = {}
            for loc in bond_locs[0]:
                bonded_atoms[loc] = atoms[loc]

            # a list of the indices in bonded_atoms for which the element is 'H'
            H_loc = [key for key, val in bonded_atoms.items() if val == "H"]

            # if that list isn't empty (so the heavy atom is bound to at least one 'H')
            if H_loc != []:
                # that central atom can be saved as a coupling heavy atom
                coupling_heavy = int(atom_ind + 1)
                # its associated H is saved (we only need the first 'H', in case there are two)
                coupling_H = int(H_loc[0] + 1)

                # to find the remaining heavy atom adjacent to the coupling atom, go through the dict.
                for key in bonded_atoms.keys():
                    # and find the first atom that is not a hydrogen
                    if bonded_atoms[key] != "H":
                        coupling_adj = int(key + 1)

                        # save as a tuple
                        coupling_sites.append(
                            (coupling_heavy, coupling_H, coupling_adj)
                        )
                        break  # we only need one adjacent atom

    return coupling_sites

In [9]:
def build_graph(
    conmat: np.ndarray,  # connectivity matrix
    atoms: list,  # element type for each atom
    atnum: list,  # atomic numbers for each atom
):

    """
    Construct a NetworkX graph where the nodes are heavy atoms and the edge weights are
    the inverse product of the atomic numbers of the nodes connected by that edge.

    :param conmat: connectivity matrix (2D array)
    :param atoms: ordered list of element types for each atom
    :param atnum: ordered list of atomic numbers for each atom

    :return G: NetworkX graph
    """

    node_labels = []

    for i in range(0, len(atoms)):
        if (
            atoms[i] != "H"
        ):  # don't add H atoms to connectivity matrix; to reduce the size of the graph
            node_labels.append(
                int(i + 1)
            )  # to keep the original xyz indices in the node labels
        else:
            pass

    G = (
        nx.OrderedGraph()
    )  # generate an ordered graph with NetworkX, otherwise the nodes are labelled randomly
    G.add_nodes_from(node_labels)  # only place nodes for heavy atoms, as defined above

    for i in node_labels:
        for j in node_labels[1:]:  # start with the second index for j
            if (
                conmat[i - 1, j - 1] == 1.0
            ):  # indexing of conmat starts at 0, while atom labels start at 1
                edgeweight = (
                    atnum[i - 1] * atnum[j - 1]
                )  # if there's a bond between two heavy atoms, add an edge between their nodes
                G.add_edge(
                    i, j, weight=edgeweight
                )  # and assign to that edge a weight as the product of their atomic numbers
            else:
                pass
    return G

In [10]:
def resistance_pairs(
    G: networkx.classes.ordered.OrderedGraph,  # the graph to use
    central_atom: int,  # the atom to analyze
    central_atom_index: int,  # its index in the list of nodes
    neighbor_list: list,  # the neighbors whose resistance distance to the central atom we will calculate
    resist_mat: np.ndarray,  # the current (incomplete) matrix of all resistances
    resist_list: list,  # the current (incomplete) list of resistances involving the central atom
):

    """
    Compute resistance between a given central node and all the nodes in a list of neighbors.

    :param G: NetworkX graph of heavy atoms
    :param central_atom: node whose resistances with its neighbors of a given degree (neighbor_list) will be computed
    :param central_atom_index: the index of central_atom in the list of nodes
    :param neighbor_list: list of nodes whose resistances will be computed with respect to central_atom
    :param resist_mat: current (uncomplete) 2D array of pairwise resistances that have been computed so far for this graph
    :param resist_list: current (uncomplete) list of resistances involving central_atom so far

    :return resist_mat: 2D array of pairwise resistances that have been computed so far for this graph
    :return resist_list: updated list of resistances involving central_atom, with new resistances added
    """

    for neigh in neighbor_list:
        if neigh == central_atom:
            pass  # skip this calculation of resistance for the very first atom with itself
        else:
            neigh_index = list(G.nodes).index(neigh)  # find the index of the neighbor
            if (
                resist_mat[central_atom_index, neigh_index] != 0.0
            ):  # if resistance between two atoms has already been saved
                res = resist_mat[central_atom_index, neigh_index]  # collect it
                resist_list.append(
                    res
                )  # and add it to the list of resistances of the central atom
            else:  # otherwise calculate it, and then save it, and then append it
                res = nx.resistance_distance(
                    G, central_atom, neigh, weight="weight", invert_weight=True
                )
                # truncate the decimals so that when the lists are compared, there will be no false divergence
                res = float("{:.7f}".format(res))
                resist_mat[central_atom_index, neigh_index] = resist_mat[
                    neigh_index, central_atom_index
                ] = res
                resist_list.append(res)

    return (
        resist_mat,
        resist_list,
    )  # updated resistance matrix and list of resistances of the central atom

In [11]:
def find_equivalent_sites(
    G: networkx.classes.ordered.OrderedGraph,  # graph to use
    topological_sites: list,  # list of sites whose topologies we will compare
):

    """
    Classify nodes listed in topological_sites by their topological equivalence.

    :param G: NetworkX graph of heavy atoms
    :param topological_sites: list of sites whose topologies will be compared

    :return equivalencies: list of lists grouping all atoms of the same topology together
    """

    nodelist = list(G.nodes)  # make a list of the nodes in the graph
    resist_mat = np.zeros(
        (len(G.nodes), len(G.nodes))
    )  # empty matrix where resistances will be stored
    equivalencies = [
        [] for i in range(len(G.nodes))
    ]  # list of lists (one list per atom) where equivalencies will be stored

    for i in range(len(topological_sites)):  # all atoms to screen for equivalence

        atom_A = topological_sites[i]  # the first atom in question
        atom_A_index = nodelist.index(atom_A)  # the index of that atom

        # compute topology only if this atom has not already been marked as equivalent to something else
        if not any(atom_A in atoms for atoms in equivalencies):
            equivalencies[atom_A_index].append(
                atom_A
            )  # add this atom to its own equivalence list
            for j in range(
                i + 1, len(topological_sites)
            ):  # loop through all other atoms with larger indices

                atom_B = topological_sites[j]  # atom to compare to atom A
                atom_B_index = nodelist.index(atom_B)  # index of that atom

                # compute topology only if this atom has not already been marked as equivalent to something else
                if not any(atom_B in atoms for atoms in equivalencies):

                    resist_list_A = []
                    last_neigh_A = []
                    resist_list_B = []
                    last_neigh_B = []

                    # increasing the generations of neighbors until either all atoms are covered or the lists of resistances diverge
                    for step in range(0, len(nodelist)):
                        # all of the atoms connected to atoms A and B by up to 'step' number of edges
                        curr_neigh_A = list(
                            nx.single_source_shortest_path_length(
                                G, atom_A, cutoff=step
                            )
                        )
                        curr_neigh_B = list(
                            nx.single_source_shortest_path_length(
                                G, atom_B, cutoff=step
                            )
                        )

                        # if all possible atoms have been considered (and so increasing step size no longer increases # of neighbors
                        if last_neigh_A == curr_neigh_A:
                            # print(atom_A,atom_B, " are equivalent, since we've computed all possible generations\n")
                            equivalencies[atom_A_index].append(
                                atom_B
                            )  # place atom B into the list of equivalencies of atom A
                            break  # stop the for loop for atom B

                        else:
                            # take just new neighbors that weren't in the previous step size
                            nextgen_neigh_A = [
                                neigh
                                for neigh in curr_neigh_A
                                if neigh not in last_neigh_A
                            ]
                            nextgen_neigh_B = [
                                neigh
                                for neigh in curr_neigh_B
                                if neigh not in last_neigh_B
                            ]
                            # the current neighbors now become the last set of neighbor for the next step of the for loop
                            last_neigh_A = curr_neigh_A
                            last_neigh_B = curr_neigh_B

                            # compute resistances between center nodes and all neighbors in the current generation
                            resist_mat, resist_list_A = resistance_pairs(
                                G,
                                atom_A,
                                atom_A_index,
                                nextgen_neigh_A,
                                resist_mat,
                                resist_list_A,
                            )
                            resist_mat, resist_list_B = resistance_pairs(
                                G,
                                atom_B,
                                atom_B_index,
                                nextgen_neigh_B,
                                resist_mat,
                                resist_list_B,
                            )

                            # if the lists are the same, atoms A and B have the same topology up to this generation
                            if Counter(resist_list_A) == Counter(resist_list_B):
                                continue

                            # if the lists diverge, A and B are different, so we can stop the comparison
                            else:
                                # print(atom_A,atom_B, " are different by generation {}\n".format(step))
                                break

    return equivalencies

In [12]:
class coupling_frag(object):

    """
    Define the coupling_frag class, which contains all the relevant geometric and topological information
    about a given molecule.
    """

    def __init__(self, name, atoms, coord, coupling_sites, nonequiv_sites):
        self.name: str = name
        self.atoms: list = atoms
        self.coord: np.ndarray = coord
        self.coupling_sites: list = coupling_sites
        self.nonequiv_sites: list = nonequiv_sites

    def print_name(self):
        print(self.name)

In [13]:
def build_frag(
    path_to_xyz,
    factor=1.0,
    atoms=None,
    coords=None,
    n_atoms=None,
    name=None,
    coupling_sites=None,
    nonequiv_sites=None,
):

    """
    Prepare a fragment object from an xyz file.

    :param path_to_xyz: path to the xyz file from which a coupling_fragment object is generated
    :param factor: value by with the sum of covalent radii of atom pairs is multiplied to determine bonding when generating the connectivity matrix

    :return frag: a coupling_frag object
    """

    # load XYZ coordinates, atom types, molecule size and name (based on xyz file name)
    if atoms is None and coords is None and n_atoms is None and name is None:
        atoms, coord, n_atoms, name = read_geom(path_to_xyz)

    # generate atomic radii and atmoic number ofr each atom
    radii, atnum = genradii(atoms)

    # generate connectivity matrix, number of connection sites per atom, and laplacian
    # conmat, connec, laplacian = genconmat(n_atoms, atoms, coord, radii, factor) # generate connectivity information with homemade code; use factor=1.3
    conmat, connec, laplacian = genconmat_ase(
        path_to_xyz, factor
    )  # generate connectivity information with ase; use factor=1.0

    # gives a list of arrays as (coupling_heavy, coupling_H, coupling_adj)
    if coupling_sites is None:
        coupling_sites = identify_coupling_sites(atoms, conmat, connec)

    if nonequiv_sites is None:

        # generate NetworkX graph using the connectivity found in the xyz structure
        G = build_graph(conmat, atoms, atnum)

        # the sites whose topologies we are interested in are those which have been identifed as coupling sites
        topological_sites = [x[0] for x in coupling_sites]

        # classify nodes listed under topological_sites by their topological equivalence
        equivalencies = find_equivalent_sites(G, topological_sites)

        # to get just one coupling site of each topology, go through the coupling sites found by identify_coupling_sites
        # and takes one from each list of equivalent sites as determined by find_equivalent_sites

        one_equivalence = (
            []
        )  # a list of lists containing all the topologically equivalent atoms
        nonequiv_sites = (
            []
        )  # a list of tuples of all topologically unique coupling sites (only used when looking for coupling sites)

        for atom_list in equivalencies:
            if atom_list != []:  # remove the empty lists
                first_instance = atom_list[0]
                one_equivalence.append(first_instance)
                new_ind = topological_sites.index(
                    first_instance
                )  # only use these two lines for collecting non-equivalent
                nonequiv_sites.append(
                    coupling_sites[new_ind]
                )  # coupling sites; otherwise just use the one_equivalence list

    # build a coupling_frag object
    frag = coupling_frag(name, atoms, coord, coupling_sites, nonequiv_sites)

    return frag

In [14]:
def rot_in_x(theta: float, input_geom: np.ndarray):

    """
    Rotate coordinates about x-axis with a simple rotation matrix.

    :param theta: angle by which to rotate coordinates along x-axis
    :param input_geom: coordinates to rotate (2D array)
    :return output_geom: coordinates following rotation (2D array)
    """

    c, s = math.cos(theta), math.sin(theta)
    rotx = np.array([[1.0, 0, 0], [0, c, -s], [0, s, c]])
    output_geom = np.dot(input_geom, rotx)
    return output_geom

In [15]:
def rot_in_y(theta: float, input_geom: np.ndarray):

    """
    Rotate coordinates about y-axis with a simple rotation matrix.

    :param theta: angle by which to rotate coordinates along y-axis
    :param input_geom: coordinates to rotate (2D array)
    :return output_geom: coordinates following rotation (2D array)
    """

    c, s = math.cos(theta), math.sin(theta)
    roty = np.array([[c, 0, s], [0, 1.0, 0], [-s, 0, c]])
    output_geom = np.dot(input_geom, roty)
    return output_geom

In [16]:
def rot_in_z(theta: float, input_geom: np.ndarray):

    """
    Rotate coordinates about z-axis with a simple rotation matrix.

    :param theta: angle by which to rotate coordinates along z-axis
    :param input_geom: coordinates to rotate (2D array)

    :return output_geom: coordinates following rotation (2D array)
    """

    c, s = math.cos(theta), math.sin(theta)
    rotz = np.array([[c, -s, 0], [s, c, 0], [0, 0, 1.0]])
    output_geom = np.dot(input_geom, rotz)
    return output_geom

In [17]:
def align_mol1(frag, site_index: int):

    """
    Align the first fragment such that its heavy atom is at the (0,0,0) origin, the coupling H is on the x-axis,
    and the adjacent heavy atom is in the xy plane.

    :param frag: coupling_frag object
    :param site_index: index of the coupling site (in the coupling_frag object) at which the fragment is being aligned

    :return coord_3: aligned coordinates for frag (2D array)
    """

    coord = frag.coord
    coupling_heavy = frag.nonequiv_sites[site_index][0]
    coupling_H = frag.nonequiv_sites[site_index][1]
    coupling_adj = frag.nonequiv_sites[site_index][2]

    # the -1 matches the XYZ index with python indexing
    coord_cen = (
        coord - coord[int(coupling_heavy) - 1]
    )  # place the heavy atom at the [0,0,0] origin
    H = coord_cen[int(coupling_H) - 1]  # position of the H to couple

    Hdist = np.linalg.norm(
        H
    )  # Euclidean distance between initial H atom and the [0,0,0] origin
    Htest = np.array([Hdist, 0, 0])  # 'new' position of the hydrogen -> just to check

    Hxz = np.array([H[0], 0, H[2]])  # projection of H into xz plane (y=0)
    Hxz = Hxz / np.linalg.norm(Hxz)  # normalize
    Hxz_dot = np.dot(
        Hxz, np.array([1, 0, 0])
    )  # [1,0,0] is the normalized desired H position on the x-axis
    y_theta = np.arccos(
        Hxz_dot
    )  # angle by which to rotate into the xy plane (about y axis)
    coord_1 = rot_in_y(y_theta, coord_cen)  # rotate in y axis, coord -> coord_1

    Hnew = coord_1[int(coupling_H) - 1]  # update position of H

    if (
        abs(Hnew[2]) < 10**-10
    ):  # by rotating H into the xy plane, the z component of the vector should now be = 0
        pass

    else:
        coord_1 = rot_in_y(
            -2 * y_theta, coord_1
        )  # otherwise move twice theta in the other direction
        Hnew = coord_1[int(coupling_H) - 1]  #####

    H = Hnew  # update position of H
    H = H / np.linalg.norm(H)  # normalize
    H_dot = np.dot(
        H, np.array([1, 0, 0])
    )  # dot product of normed vector of H with the unitary x-axis vector
    z_theta = np.arccos(
        H_dot
    )  # angle by which to rotate about z axis so H is on the x-axis
    coord_2 = rot_in_z(z_theta, coord_1)  # rotate in z-axis, coord_1 -> coord_2

    Hnew = coord_2[int(coupling_H) - 1]  # update position of H

    if (
        abs(Hnew[1]) < 10**-10
    ):  # by rotating H into the x-axis, the y component of the vector should now be = 0
        pass

    else:
        coord_2 = rot_in_z(
            -2 * z_theta, coord_2
        )  # otherwise move twice theta in the other direction
        Hnew = coord_2[int(coupling_H) - 1]

    adj = coord_2[
        int(coupling_adj) - 1
    ]  # coordinates of the atom adjacent to the coupling position
    adj = np.array([0, adj[1], adj[2]])  # projection of adj position into yz plane
    adj = adj / np.linalg.norm(adj)  # normalize
    adj_dot = np.dot(
        adj, np.array([0, 0, 1])
    )  # dot product of normed vector of adj. heavy atom with the z-axis
    x_theta = np.arccos(
        adj_dot
    )  # rotate by this angle in the x-axis so that adj. atom is on xz plane
    coord_3 = rot_in_x(x_theta, coord_2)  # rotate in x-axis, coord_2 -> coord_3

    adjnew = coord_3[int(coupling_adj) - 1]  # update position of adj. atom

    if (
        abs(adjnew[1]) < 10**-10
    ):  # by rotating adj. into the xz plane, the y component of the vector should be = 0
        pass

    else:
        coord_3 = rot_in_x(
            -2 * x_theta, coord_3
        )  # otherwise move twice theta in the other direction
        adjnew = coord_3[int(coupling_adj) - 1]

    return coord_3

In [18]:
def align_mol2(frag, site_index: int, coupling_distance: float):

    """
    Align the second fragment such that its heavy atom is at an average sp2-sp2 bond distance from the heavy atom of the first fragment,
    the C-H C-H bonds from the two fragments form a line in the x-axis, and the adjacent heavy atom is in the xy plane.

    :param frag: coupling_frag object
    :param site_index: index of the coupling site (in the coupling_frag object) at which the fragment is being aligned
    :param coupling_distance: length of the sp2-sp2 bond to be formed, in Angstrom
    :return coord_3: aligned coordinates for frag (2D array)
    """

    coord = frag.coord
    coupling_heavy = frag.nonequiv_sites[site_index][0]
    coupling_H = frag.nonequiv_sites[site_index][1]
    coupling_adj = frag.nonequiv_sites[site_index][2]

    coord_cen = coord - coord[int(coupling_heavy) - 1]
    H = coord_cen[int(coupling_H) - 1]

    Hxz = np.array([H[0], 0, H[2]])
    Hxz = Hxz / np.linalg.norm(Hxz)
    Hxz_dot = np.dot(
        Hxz, np.array([-1, 0, 0])
    )  # [-1,0,0] is the normalized vector of the target H position for mol2
    y_theta = np.arccos(Hxz_dot)
    coord_1 = rot_in_y(y_theta, coord_cen)

    Hnew = coord_1[int(coupling_H) - 1]

    if abs(Hnew[2]) < 10**-10:
        pass

    else:
        coord_1 = rot_in_y(-2 * y_theta, coord_1)
        Hnew = coord_1[int(coupling_H) - 1]

    H = coord_1[int(coupling_H) - 1]
    H = H / np.linalg.norm(H)
    H_dot = np.dot(
        H, np.array([-1, 0, 0])
    )  # [-1,0,0] is the normalized vector of the target H position for mol2
    z_theta = np.arccos(H_dot)
    coord_2 = rot_in_z(z_theta, coord_1)

    Hnew = coord_2[int(coupling_H) - 1]

    if abs(Hnew[1]) < 10**-10:
        pass

    else:
        coord_2 = rot_in_z(-2 * z_theta, coord_2)
        Hnew = coord_2[int(coupling_H) - 1]

    adj = coord_2[int(coupling_adj) - 1]
    adj = np.array([0, adj[1], adj[2]])
    adj = adj / np.linalg.norm(adj)
    adj_dot = np.dot(adj, np.array([0, 0, 1]))
    x_theta = np.arccos(adj_dot)
    coord_3 = rot_in_x(x_theta, coord_2)

    adjnew = coord_3[int(coupling_adj) - 1]

    if abs(adjnew[1]) < 10**-10:
        pass

    else:
        coord_3 = rot_in_x(-2 * x_theta, coord_3)
        adjnew = coord_3[int(coupling_adj) - 1]

    coord_3 = coord_3 + np.array([coupling_distance, 0.0, 0.0])

    return coord_3

In [19]:
def minimal_distance(geom1: np.ndarray, geom2: np.ndarray):

    """
    Collect all the interatomic distances between pairs of atoms from each fragment and returns the second smallest of these.

    :param geom1: aligned cartesian coordinates of the first fragment involved in coupling (2D array)
    :param geom2: aligned cartesian coordinates of the second fragment involved in coupling (2D array)
    :return min_d: the second smallest distance between the two fragments (the smallest will be the coupling bond formed
    """

    interatomic_d = []
    for a in range(len(geom1)):
        for b in range(len(geom2)):
            atom1 = geom1[a]
            atom2 = geom2[b]
            dist = np.linalg.norm(atom1 - atom2)
            interatomic_d.append(dist)

    interatomic_d.remove(
        min(interatomic_d)
    )  # delete smallest distance (it should be the newly-formed bond)
    min_d = min(interatomic_d)  # collect next smallest distance

    return min_d

In [20]:
def build_dimer(
    name1: str,
    name2: str,
    dim_coord: np.ndarray,
    dim_atoms: list,
    dim_n_atoms: int,
    site1: int,
    site2: int,
    dih_sites: list,
    counter: int,
):

    """
    Write the name, coordinates, coupling data and dihedral information of the created dimer to a new xyz file.

    :param name1: name of first coupling fragment
    :param name2: name of second coupling fragment
    :param dim_coord: coordinates of newly-created dimer (2D array)
    :param dim_atoms: list of atom element types in dimer
    :param dim_n_atoms: number of atoms in the dimer
    :param site1: label of atom from first fragment involved in coupling
    :param site2: label of atom from second fragment involved in coupling
    :param dih_sites: list of labels of four atoms which form the dihedral angle (adjacent atom frag1, coupling atom frag1, coupling atom frag2, adjacent atom frag2)
    :param counter: keeps track of how many dimers have already been created from this particular monomer pair
    :return dim_name: name of the dimer which has been written to an xyz file (inc. names, coupling sites and counter)
    """

    dim_name = str(
        name1 + "_" + name2 + "_" + str(site1) + "_" + str(site2) + "_" + str(counter)
    )
    output = open(dim_name + ".xyz", "w")

    output.write(str(dim_n_atoms))  # n_atoms1 + n_atoms2
    output.write("\n")

    # print coupling information to the free 2nd line of xyz file

    output.write(
        "Name: "
        + str(dim_name)
        + " Afrag: "
        + name1
        + " Apos: "
        + str(site1)
        + " Bfrag: "
        + name2
        + " Bpos: "
        + str(site2)
        + " Dihedral atoms: "
        + str(dih_sites[0])
        + " "
        + str(dih_sites[1])
        + " "
        + str(dih_sites[2])
        + " "
        + str(dih_sites[3])
    )
    output.write("\n")

    for i, row in enumerate(dim_coord):
        outstring = dim_atoms[i]
        if (
            row[0] < 0
        ):  # corrects spacing when there is a minus sign in front of the coordinate value
            outstring += "        " + "%.5f" % row[0]
        else:
            outstring += "         " + "%.5f" % row[0]
        if row[1] < 0:
            outstring += "        " + "%.5f" % row[1]
        else:
            outstring += "         " + "%.5f" % row[1]
        if row[2] < 0:
            outstring += "        " + "%.5f" % row[2]
        else:
            outstring += "         " + "%.5f" % row[2]

        output.write(outstring)
        output.write("\n")

    output.close()
    return dim_name

In [21]:
def write_dih(dim_name: str, dih_sites: list):

    """
    Write the four atom labels forming the dihedral between the two fragments in the new dimer to a .dih file.

    :param dim_name: name of the new dimer
    :param dih_sites: list of the four atoms around the newly coupling bond

    :return: no value
    """

    output = open(dim_name + ".dih", "w")
    outstring = (
        str(dih_sites[0])
        + " "
        + str(dih_sites[1])
        + " "
        + str(dih_sites[2])
        + " "
        + str(dih_sites[3])
    )
    output.write(outstring)
    output.close()

In [22]:
def write_atlist(dim_name: str, atoms_by_frag: list):

    """
    Write the new atom labels (as they appear in the dimer), grouped by the fragments from which they originate, to an .atl file.

    :param dim_name: name of the new dimer
    :param atoms_by_frag: list of lists grouping atom labels by the fragments to which they belong

    :return: no value
    """

    output = open(dim_name + ".atl", "w")
    outstring = str(atoms_by_frag)
    output.write(outstring)
    output.close()

In [23]:
def dihedral_finder(fragA, fragB,
                    coordA:np.ndarray, coordB:np.ndarray,
                    siteA_index:int,siteB_index:int,
                    dim_n_atoms:int,dih_sites:list,counter:int,atoms_by_frag:list, coupling_distance:float):

    """
    Try to generate a dimer with a dihedral closest to coplanarity which does not lead to atoms from each fragment
    being less that a certain cutoff distance from one another. If an appropriate dihedral is found, generate the dimer;
    otherwise print an error message and carry on with the next possible combination of positions
    
    :param fragA: coupling_frag object for first coupling fragment
    :param fragB: coupling_frag object for second coupling fragment
    :param coordA: aligned coordinates of the first fragment (2D array)
    :param coordB: aligned coordinates of the second fragment (2D array)
    :param siteA_index: index of the coupling site (in coupling_frag object) at which fragA is being coupled
    :param siteB_index: index of the coupling site (in coupling_frag object) at which fragB is being coupled
    :param dim_n_atoms: number of atoms in the dimer
    :param dih_sites: list of the four atoms around the newly coupling bond
    :param counter: how many dimers have already been created from this particular monomer pair
    :param atoms_by_frag: list of lists grouping atom labels by the fragments to which they belong
    :param coupling_distance: length of the sp2-sp2 bond to be formed, in Angstrom

    :return counter: how many dimers have now been created from this particular monomer pair
    """

    name1 = fragA.name
    name2 = fragB.name

    # sites on the monomers where the new bond is formed

    site1 = int(fragA.nonequiv_sites[siteA_index][0])
    site2 = int(fragB.nonequiv_sites[siteB_index][0])

    # remove the H at the sites where the new Csp2 - Csp2 bond is being formed in the coordinate arrays

    coordA_4 = np.delete(coordA, (int(fragA.nonequiv_sites[siteA_index][1])-1), axis=0)
    coordB_4 = np.delete(coordB, (int(fragB.nonequiv_sites[siteB_index][1])-1), axis=0)

    # fragment atom lists with hydrogen at coupling site removed

    atoms1_new = fragA.atoms[:int(fragA.nonequiv_sites[siteA_index][1])-1] + fragA.atoms[int(fragA.nonequiv_sites[siteA_index][1]):]
    atoms2_new = fragB.atoms[:int(fragB.nonequiv_sites[siteB_index][1])-1] + fragB.atoms[int(fragB.nonequiv_sites[siteB_index][1]):]

    for m in [0, 16, 1, 15, 2, 14, 3, 13, 4, 12, 5, 11, 6, 10, 7, 9, 8]: # &&&& starting wiht coplanar geometries and moving to orthogonal ones (0/16 to 8)
    #for m in [8, 7, 9, 6, 10, 5, 11, 4, 12, 3, 13, 2, 14, 1, 15, 0, 16]: # &&&& do the opposite here (8 to 0/16, orthogonal to coplanar)

        for n in [1,-1]:
            x_theta = ((m*n/16)*np.pi) # search possible dihedral angles between frags, starting with planar geometries
            coord_temp = rot_in_x(x_theta, coordB_4) # rotate second frag to create the dihedral
            min_d = minimal_distance(coordA_4, coord_temp) # give the minimum distance between any one atom of each frag

            if min_d >= 1.5: # %%%% distance limit for what is considered a steric clash between fragments; make sure it's at least larger than the bond being formed!
                dim_coord = np.append(coordA_4, coord_temp , axis=0) # concatenate xyz coords of 1st and 2nd frags
                dim_atoms = atoms1_new + atoms2_new # concatenate lists of atom elements
                counter = counter + 1 # only increase the index in the dimer name if a new dimer can be generated
                dim_name = build_dimer(name1,name2,dim_coord,dim_atoms,dim_n_atoms,site1,site2,dih_sites,counter) #build the dimer
                write_dih(dim_name,dih_sites) # make a separate .dih file containing the indices of the four atoms forming the dihedral between the fragments
                write_atlist(dim_name,atoms_by_frag) # make a separate .atl file containing the indices of the atoms in each fragment 
                print("Dimer generated: {0} with dihedral {1} degs between new atoms {2} at initial fragment A pos {3} and B pos {4}"
                      .format(dim_name,str(x_theta*(180/np.pi))[:5],dih_sites,site1,site2)) # prints summary of coupling to terminal (all important information also in .xyz, .atl and .dih files)
                break

            elif m == 8 and n == -1: # &&&& this is the last m,n combination to check whwne starting with coplanar structures; give failure message
            #elif m == 16 and n == -1: # &&&& this is the last m,n combination to check when starting with orthogonal structures; give failure message
                print("Could not find a reasonable geometry for fragments A {0} and B {1} at A pos {2} and B pos {3} --no dimer generated"
                      .format(name1,name2,site1,site2))
            else:
                pass

        if min_d >= 1.5: # check same condition for outer loop once inner loop has been broken %%%%
            break

    return counter


In [24]:
def dih_label(fragA, siteA_index: int, fragB, siteB_index: int):

    """
    Prepare a list of the four atoms used to calculate the dihedral formed by the new sp2-sp2 bond.

    :param fragA: coupling_frag object for first coupling fragment
    :param fragB: coupling_frag object for second coupling fragment
    :param siteA_index: index of the coupling site (in coupling_frag object) at which fragA is being coupled
    :param siteB_index: index of the coupling site (in coupling_frag object) at which fragB is being coupled

    :return dih_sites: list of labels of four atoms which form the dihedral angle (adjacent atom frag1, coupling atom frag1, coupling atom frag2, adjacent atom frag2)
    """

    coupling_heavy1 = fragA.nonequiv_sites[siteA_index][0]
    coupling_H1 = fragA.nonequiv_sites[siteA_index][1]
    coupling_adj1 = fragA.nonequiv_sites[siteA_index][2]
    atoms1 = fragA.atoms

    coupling_heavy2 = fragB.nonequiv_sites[siteB_index][0]
    coupling_H2 = fragB.nonequiv_sites[siteB_index][1]
    coupling_adj2 = fragB.nonequiv_sites[siteB_index][2]

    # compares the indices of the heavy atoms with those of the H atoms which have been removed
    # to establish the indices of the atoms describing the dihedral formed between the two fragments of the new dimer

    # because H will be removed, the indices of the heavy and adjacent atoms of mol1
    # have to be corrected if the index of H is smaller than the others

    if coupling_adj1 > coupling_H1:
        dih_1 = int(coupling_adj1) - 1
    else:
        dih_1 = int(coupling_adj1)

    if coupling_heavy1 > coupling_H1:
        dih_2 = int(coupling_heavy1) - 1
    else:
        dih_2 = int(coupling_heavy1)

    # correct the the indices of the heavy and adjacent atoms on mol2 in the same way, adding the length of mol1

    if coupling_adj2 > coupling_H2:
        dih_4 = int(coupling_adj2) + len(atoms1) - 2
    else:
        dih_4 = int(coupling_adj2) + len(atoms1) - 1

    if coupling_heavy2 > coupling_H2:
        dih_3 = int(coupling_heavy2) + len(atoms1) - 2
    else:
        dih_3 = int(coupling_heavy2) + len(atoms1) - 1

    # save these values as the four indices forming the dihedral in the new dimer

    dih_sites = [dih_1, dih_2, dih_3, dih_4]
    return dih_sites

In [25]:
def coupling(
    fragA, fragB, coupling_distance=1.35
):  # !!!! translate along x-axis by average bondlength; change this depending on the bonds being formed

    """
    Couple fragments A and B together in all of the possible non-equivalent coupling sites

    :param fragA: coupling_frag object for first coupling fragment
    :param fragB: coupling_frag object for second coupling fragment
    :param coupling_distance: length of the sp2-sp2 bond to be formed, in Angstrom

    :return: no value
    """

    n_atoms1 = len(fragA.atoms)
    n_atoms2 = len(fragB.atoms)

    dim_n_atoms = n_atoms1 + n_atoms2 - 2

    atoms_in_frag1 = [i for i in range(1, n_atoms1, 1)]
    atoms_in_frag2 = [i for i in range(n_atoms1, dim_n_atoms + 1, 1)]

    atoms_by_frag = [atoms_in_frag1, atoms_in_frag2]

    print(
        "Creating dimers from fragA {0} and fragB {1}; atoms per fragment: ".format(
            fragA.name, fragB.name
        )
    )
    print(atoms_by_frag)

    # keep track of how many structural isomers of each dimer have been created
    counter = 0

    # loop through all possible coupling sites in each fragment, as defined by the bnd files
    for i in range(len(fragA.nonequiv_sites)):
        for j in range(len(fragB.nonequiv_sites)):

            # align the fragments

            coordA = align_mol1(fragA, i)
            coordB = align_mol2(fragB, j, coupling_distance)

            # find the new labels of atoms that will make up the dihedral

            dih_sites = dih_label(fragA, i, fragB, j)

            # for this i,j combination of coupling positions, search for a geometry where there is no steric clash
            # if an appropriate geometry is found, the dimer will be created and the counter will increase by one
            # otherwise, no dimer will be created and the counter will remain the same

            counter = dihedral_finder(
                fragA,
                fragB,
                coordA,
                coordB,
                i,
                j,
                dim_n_atoms,
                dih_sites,
                counter,
                atoms_by_frag,
                coupling_distance,
            )

    print("\n")

In [26]:
####### EXAMPLE INPUT AND OUTPUT #######

In [27]:
# file that contains a list of pairs of fragments that will be coupled to one another
monomer_pair_file = open("./fragments_to_combine.txt", "r")
content = monomer_pair_file.read()
monomer_pair_file.close()
content = ast.literal_eval(content)

# path to monomer xyzs
monomer_xyz_path = "./"

orig_stdout = sys.stdout

# output file that documents which fragments were coupled and how
f = open("coupling_output.txt", "w")
sys.stdout = f

for pair in content:
    pathA = monomer_xyz_path + pair[0] + ".xyz"
    pathB = monomer_xyz_path + pair[1] + ".xyz"
    fragA = build_frag(
        path_to_xyz=pathA, factor=1.0
    )  # make the monomer coupling_frag objects
    fragB = build_frag(path_to_xyz=pathB, factor=1.0)

    coupling(fragA, fragB, 1.35)  # this line to form coupling products @@@@ !!!!

sys.stdout = orig_stdout
f.close()

/tmp/ipykernel_2843573/4152113028.py:31: DeprecationWarning: OrderedGraph is deprecated and will be removed in version 3.0.
Use `Graph` instead, which guarantees order is preserved for
Python >= 3.7

  nx.OrderedGraph()


AttributeError: module 'scipy.sparse' has no attribute 'coo_array'